In [21]:
import yaml
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import gc

In [2]:
def load_rating():
    gc.collect()
    with open(f'../data/books.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['reviewerID'], data['asin']

In [3]:
arr = []
i = 0
for data in tqdm(load_rating()):
    arr.append(data)

51311621it [09:23, 91059.19it/s] 


In [4]:
df = pd.DataFrame(arr, columns=['user','item'])

In [5]:
cnt_user = df.groupby('user').agg({'item' : 'count'})
cnt_book = df.groupby('item').agg({'user' : 'count'})

In [17]:
user_more_h = cnt_user[cnt_user['item'] > 40]
book_more_h = cnt_book[cnt_book['user'] > 40]

In [22]:
user_list = user_more_h.index
book_list = book_more_h.index

In [34]:
df_t = df.loc[df['item'].isin(book_list)]
df_t = df_t.loc[df_t['user'].isin(user_list)]

In [35]:
books = df_t['item'].unique()

In [36]:
len(books)

202233

In [37]:
def get_data(*keys):
    data = volume_info
    global err_toggle
    try :
        for key in keys:
            data = data[key]
        return data
    except :
        if key == 'authors' or key == 'categories':
            return []
        else : return None

In [19]:
book_infos = pd.DataFrame(columns=['ISBN', 'authors', 'publisher', 'publish_date', 'description', 'page_cnt', 'categories', 'average_rating', 'rating_count', 'synopsis'])

In [40]:
with open('../data/google_api_4.tsv', 'a') as f:
#    f.write(f'ISBN\tauthors\tpublisher\tpublish_date\tdescription\tpage_cnt\tcategories\taverage_rating\trating_count\tsynopsis\n')
    for i in tqdm(range(111490,len(books))) :
        ISBN = books[i]
        if 'B' in ISBN :
            continue

        response = requests.get(f"https://www.googleapis.com/books/v1/volumes?q={ISBN}")


        try : volume_info = response.json()['items'][0]['volumeInfo']
        except :
            f.write(f'{ISBN}\tNone\tNone\tNone\tNone\tNone\tNone\tNone\tNone\tNone\n')
            continue   

        try :
            if volume_info['industryIdentifiers'][0]["identifier"] != ISBN:
                f.write(f'{ISBN}\tNone\tNone\tNone\tNone\tNone\tNone\tNone\tNone\tNone\n')
                continue
        except :
            f.write(f'{ISBN}\tNone\tNone\tNone\tNone\tNone\tNone\tNone\tNone\tNone\n')
            continue


        description = get_data('description')
        authors = get_data('authors')
        publisher = get_data("publisher")
        publish_date = get_data("publishedDate")
        page_cnt = get_data('pageCount')
        categories = get_data('categories')
        average_rating = get_data('averageRating')
        rating_count = get_data('ratingsCount')
        info_link = get_data('infoLink')
        synopsis = "None"
        
        if info_link != None : 
            html = BeautifulSoup(requests.get(info_link).text)
            try : 
                synopsis = html.select_one('#synopsistext').text
            except : synopsis = "None"

        
        f.write(f'{ISBN}\t{authors}\t{publisher}\t{publish_date}\t{description}\t{page_cnt}\t{categories}\t{average_rating}\t{rating_count}\t{synopsis}\n')

100%|██████████| 90743/90743 [33:19:51<00:00,  1.32s/it]    
